In [1]:
import pyqubo
import numpy as np
from pyqubo import Array, Placeholder, Spin, solve_qubo, Constraint, OneHotEncInteger

# Knapsack Problem with Integer Weights

We have N objects, each with an integer weight and price, that we want to put inside a knapsack that can only carry weight **W**. We want to maximize the total cost of the items we collect while subject to the weight constraint. The total weight in the knapsack _W_ can be represented as

$$
W = \sum_{\alpha = 1}^{N} w_{\alpha}x_{\alpha},
$$

and its total cost _C_ as 

$$
C = \sum_{\alpha = 1}^{N} c_{\alpha}x_{\alpha},
$$

where $x_{a}$ determines whether or not object $\alpha$ is in the knapsack. The solution takes the form $\lambda H_{A} + H_{B}$. We define $H_{A}$ as

$$
H_{A} = \left(1-\sum_{n = 1}^{W} y_{n} \right)^2
+ \left (\sum_{n = 1}^{W} n y_{n} - \sum_{\alpha} w_{\alpha} x_{\alpha} \right)^2,
$$

where $y_{n}$ represents a binary that is 1 if the final weight is $n$, and 0 otherwise, and $H_{B}$ as

$$
H_{B} = \sum_{\alpha} c_{\alpha} x_{\alpha}.
$$

In [2]:
weights = [1, 3, 7, 9] 
values = [10, 2, 3, 6] 
max_weight = 10

items = Array.create('item', shape=4, vartype="BINARY")

knapsack_weight = sum(weights[i] * items[i] for i in range(len(items)))
knapsack_value = sum(values[i] * items[i] for i in range(len(items)))

lmd = Placeholder("lambda")

weight_one_hot = OneHotEncInteger("weight_one_hot", 1, max_weight, strength=lmd)
Ha = Constraint((weight_one_hot - knapsack_weight)**2, "weight_constraint")
Hb = knapsack_value
H = Ha - Hb

model = H.compile()
steps = 15
feed_dict = {'lambda': 1.0}
for i in range(steps):
    qubo, offset = model.to_qubo(feed_dict=feed_dict)
    sol = solve_qubo(qubo)
    solution, broken, energy = model.decode_solution(sol, vartype="BINARY", feed_dict=feed_dict)
    if not broken:
        break
    else: 
        feed_dict['lambda'] *= 1.2

In [3]:
weight_one_hot

WithPenalty(value=((Binary(weight_one_hot[0])*Num(0))+(Binary(weight_one_hot[1])*Num(1))+(Binary(weight_one_hot[2])*Num(2))+(Binary(weight_one_hot[3])*Num(3))+(Binary(weight_one_hot[4])*Num(4))+(Binary(weight_one_hot[5])*Num(5))+(Binary(weight_one_hot[6])*Num(6))+(Binary(weight_one_hot[7])*Num(7))+(Binary(weight_one_hot[8])*Num(8))+(Binary(weight_one_hot[9])*Num(9))+Num(1)),penalty=(Const(weight_one_hot_const, ((Binary(weight_one_hot[0])+Binary(weight_one_hot[1])+Binary(weight_one_hot[2])+Binary(weight_one_hot[3])+Binary(weight_one_hot[4])+Binary(weight_one_hot[5])+Binary(weight_one_hot[6])+Binary(weight_one_hot[7])+Binary(weight_one_hot[8])+Binary(weight_one_hot[9])+Num(-1))*(Binary(weight_one_hot[0])+Binary(weight_one_hot[1])+Binary(weight_one_hot[2])+Binary(weight_one_hot[3])+Binary(weight_one_hot[4])+Binary(weight_one_hot[5])+Binary(weight_one_hot[6])+Binary(weight_one_hot[7])+Binary(weight_one_hot[8])+Binary(weight_one_hot[9])+Num(-1))))*Placeholder(lambda)))

In [4]:
solution

{'item': {0: 1, 1: 0, 2: 0, 3: 1},
 'weight_one_hot': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 1}}

## The Knapsack Problem Using D-Wave Solvers

In [5]:
import dimod
from dwave.embedding.chain_breaks import majority_vote
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
from dwave.embedding.chimera import find_clique_embedding

In [6]:
def create_dw_sampler(endpoint, token, solver):
    dw_sampler = DWaveSampler(endpoint=endpoint, token=token, solver=solver)
    embedding = find_clique_embedding(64, m=16, n=16, t=4) 
    chains = list(embedding.values())
    clique_sampler = FixedEmbeddingComposite(dw_sampler, embedding)
    return clique_sampler, chains

In [7]:
dw_sampler, chains = create_dw_sampler(
    "https://cloud.dwavesys.com/sapi",
    "api-key",
    "DW_2000Q_VFYC_5")

In [8]:
sampler_kwargs = {
    "num_reads": 1000,
    "annealing_time": 20,
    "chains": chains,
    "postprocess": "optimization",
    "num_spin_reversal_transforms": 10,
    "auto_scale": True,
    "chain_strength": 2.0,
    "chain_break_fraction": True
}

In [9]:
def objective(feed_dict, sampler_kwargs):
    qubo, offset = model.to_qubo(index_label=True, feed_dict=feed_dict)
    normalized_bqm = dimod.BinaryQuadraticModel.from_qubo(qubo)
    normalized_bqm.normalize([-1, 1])
    response = dw_sampler.sample(normalized_bqm,  **sampler_kwargs)
    decoded_sol, broken, energy = model.decode_dimod_response(response, topk=1, feed_dict=feed_dict)[0]
    return decoded_sol, broken, energy

In [10]:
steps=10
feed_dict = {"lambda": 1.0}
mu = 1.1
for i in range(steps):
    decoded_sol, broken_const, energy = objective(feed_dict, sampler_kwargs)
    if not broken_const:
        break
    feed_dict['lambda'] *= mu

In [11]:
decoded_sol

{'item': {0: 1, 1: 0, 2: 0, 3: 1},
 'weight_one_hot': {0: 0,
  1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 0,
  7: 0,
  8: 0,
  9: 1}}